In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
bs=52 # 52 - Jeremey, 20 - default

In [4]:
bptt= 70 #70 - Jeremey, 35 - default

In [5]:
def create_data_lm(tokens,bs):
    import itertools
    # Collapse into a single large array
    tokens=np.asarray(list (itertools.chain(*tokens)))
    # How many batches
    n_batch=len(tokens)//bs
    # Truncate to exclude the ones at the end
    tokens=tokens[:bs*n_batch]
    # Reshape
    tokens=tokens.reshape(bs,-1)
    return tokens

In [6]:
train_tokens=create_data_lm(df_train['tokens'],bs)
valid_tokens=create_data_lm(df_valid['tokens'],bs)
test_tokens=create_data_lm(df_test['tokens'],bs)
train_tokens.shape, valid_tokens.shape, test_tokens.shape

((52, 396650), (52, 43895), (52, 143259))

In [7]:
def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [8]:
n_batch=train_tokens.shape[1]
for i in range(0,n_batch,bptt):
    seq_len=min(bptt,n_batch-1-i)
    x=train_tokens[:,i:i+seq_len]
    y=train_tokens[:,i+1:i+1+seq_len]
    print (x.shape, y.shape)
    break

(52, 70) (52, 70)


In [9]:
x.shape,y.shape

((52, 70), (52, 70))

In [10]:
x,y

(array([[   41,    42,    43,    40, ...,     2,  1896,   463,     4],
        [ 1071,    23,    67,    39, ...,     2,    98,    11,    63],
        [   54,    34,     7,   190, ...,     2,    87,    34,  5998],
        [    7,  1213,     8,   453, ...,   406,     4,     2,    46],
        ...,
        [   60,   126,   931,     6, ...,   114,    39,   599,    63],
        [    2, 15094,   132,    65, ...,    56,  1350,   209,   281],
        [11783,   658,   318,     3, ...,     6,    64,    79,  1268],
        [   42,    43,    40,     2, ...,   708,   111,     4,     2]]),
 array([[   42,    43,    40,    13, ...,  1896,   463,     4,     2],
        [   23,    67,    39,    18, ...,    98,    11,    63,    28],
        [   34,     7,   190,  1732, ...,    87,    34,  5998,    47],
        [ 1213,     8,   453,    21, ...,     4,     2,    46,     7],
        ...,
        [  126,   931,     6,   126, ...,    39,   599,    63,   314],
        [15094,   132,    65,   411, ...,  1350, 

In [11]:
x.shape,y.shape

((52, 70), (52, 70))

In [12]:
train_tokens.shape

(52, 396650)

## pre-trained AWD-LSTM

In [13]:
PATH_AWD_LSTM='/home/kirana/Documents/phd/data/pre-trained/awd_lstm/'

In [14]:
ls {PATH_AWD_LSTM}

bwd_wt103_enc.h5  fwd_wt103_enc.h5  itos_wt103.pkl  wt103_60002/
bwd_wt103.h5      fwd_wt103.h5      wt103_238642/   wt103_tiny.tgz


In [15]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103.h5', map_location=lambda storage, loc: storage)

In [16]:
wgts

OrderedDict([('0.encoder.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                        1.9580e-02,  1.8548e-01],
                      [ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
                        2.1776e-05, -1.2394e-05],
                      [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
                       -8.1352e-02,  1.8054e-01],
                      ...,
                      [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
                        4.1632e-03, -1.3171e-03],
                      [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
                        4.6596e-03, -2.5850e-03],
                      [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
                        5.8085e-03, -1.8940e-03]])),
             ('0.encoder_with_dropout.embed.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                

In [17]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103_enc.h5', map_location=lambda storage, loc: storage)

In [18]:
wgts

OrderedDict([('encoder.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                        1.9580e-02,  1.8548e-01],
                      [ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
                        2.1776e-05, -1.2394e-05],
                      [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
                       -8.1352e-02,  1.8054e-01],
                      ...,
                      [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
                        4.1632e-03, -1.3171e-03],
                      [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
                        4.6596e-03, -2.5850e-03],
                      [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
                        5.8085e-03, -1.8940e-03]])),
             ('encoder_with_dropout.embed.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                    

In [19]:
itos2=pickle.load(open(f'{PATH_AWD_LSTM}/itos_wt103.pkl','rb'))
stoi2 = collections.defaultdict(lambda: -1, { v: k for k, v in enumerate(itos2) })

In [20]:
wgts['encoder.weight'].shape

torch.Size([238462, 400])

In [21]:
enc_wgts = wgts['encoder.weight'].numpy() # converts np.ndarray from torch.FloatTensor.output shape: (238462, 400)
row_m = enc_wgts.mean(0) # returns the average of the array elements along axis 0. output shape: (400,)

In [22]:
enc_wgts.shape, len(itos)

((238462, 400), 59972)

In [23]:
n_emb=enc_wgts.shape[1]

In [24]:
new_w = np.zeros((len(itos),n_emb), dtype=np.float32) # shape: (60002, 400)

for i, w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r >= 0 else row_m

In [25]:
len(set(itos).difference(set(itos2)))

8910

## Model Architecture

In [26]:
from torch import nn
import torch
from torch.autograd import Variable

In [27]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400 #650
n_layers=2
dropout=0.2
wd=1e-7

In [28]:
class language_model (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,adaptive_log_softmax=True,tie_weights=True):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx
        self.adaptive_log_softmax,self.tie_weights=adaptive_log_softmax,tie_weights
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        self.gen_hidden()
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
            
        if self.adaptive_log_softmax is False:
            self.criterion=nn.CrossEntropyLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer
        self.encoder=nn.Embedding(self.n_inp,self.n_emb)
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=False)
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        if self.adaptive_log_softmax:
            # Adaptive Log Softmax Loss
            self.adaptive_softmax=AdaptiveLogSoftmaxWithLoss(self.n_hidden,
                                    self.n_inp,
                                    cutoffs=[round(self.n_inp/15),3*round(self.n_inp/15)],
                                    div_value=4,
                                    get_full_prob=True)
        else:
            self.decoder=nn.Linear(self.n_hidden,self.n_inp)
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
        if self.tie_weights:
            self.decoder.weight.requires_grad=False
        
    
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True
        if self.tie_weights:
            self.decoder.weight.requires_grad=True
        
    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
        if self.tie_weights:
            self.decoder.weight=self.encoder.weight
    
    def gen_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb):
        #import pdb
        #pdb.set_trace()
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        out,new_hidden=self.lstm(embs,self.hidden)
        out=self.dropout_op(out)
         # Wrap the hidden state in a new tensor without the gradients
        self.hidden=(Variable(new_hidden[0].data,requires_grad=False).to(self.device),\
                     Variable(new_hidden[1].data,requires_grad=False).to(self.device))
        if self.adaptive_log_softmax:
            out=out.reshape(out.size(0)*out.size(1),out.size(2))        # output is of shape n_batch * n_seq * n_hidden
      
            out=self.adaptive_softmax(out,Yb.view(-1))
            loss=out.loss
            preds=out.output_full
        else:
            #import pdb
            #pdb.set_trace()
            preds=self.decoder(out.contiguous().view(out.size(0)*out.size(1), out.size(2)))
            loss=self.criterion(preds,Yb.contiguous().view(-1))

        return preds, loss  

In [29]:
def accuracy_multinomial(preds,actual):
    preds=preds.max(1)[1]
    correct=preds==actual
    return correct.float().sum()/len(correct)

In [30]:
device="cuda:0"
model=language_model(n_inp,n_emb,n_hidden,n_layers,False,bs,device,0.05,0.25,0.25,new_w,False,True)


initializing


In [31]:
new_w.shape

(59972, 400)

In [32]:
model.encoder.weight.data.shape,model.decoder.weight.data.shape

(torch.Size([59972, 400]), torch.Size([59972, 400]))

In [33]:
nn.Linear(n_hidden,n_inp).weight.data.shape, n_hidden, n_inp

(torch.Size([59972, 400]), 400, 59972)

### Test if model forward works

In [34]:
model.decoder.weight.shape,model.encoder.weight.shape

(torch.Size([59972, 400]), torch.Size([59972, 400]))

In [35]:
x.shape

(52, 70)

In [36]:
if 1==0:
    model.forward(torch.LongTensor(x),torch.LongTensor(y))

In [37]:

model=model.to(device)

## Learner

In [38]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1

    
    def fit (self,Xb,Yb,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb)
        
       
            
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        n_batch=iterator.shape[1]
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.gen_hidden()
        #for k,i in enumerate(range(0,n_batch,self.bptt)):
        n_batch=iterator.shape[1]
        while i<n_batch-bptt:
            if mode_train:
                cust_bptt=self.bptt if np.random.random() < 0.95 else self.bptt//np.random.randint (2,4)
            else:
                cust_bptt=bptt
            seq_len=min(cust_bptt,n_batch-1-i)
            Xb=train_tokens[:,i:i+seq_len]
            Yb=train_tokens[:,i+1:i+1+seq_len]
            Xb=torch.LongTensor(Xb)
            Yb=torch.LongTensor(Yb)
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
            k=k+1
            i=i+seq_len
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
        
        if 1==0:
            lr /= 4.0
            # Freeze all the layers initially
            for param in resnet.parameters():
                param.requires_grad=False
            torch.save(resnet,'resnet')
            torch.save(resnet.state_dict(),'resnet_state_dict')
            resnet.load_state_dict(torch.load('resnet_state_dict'))
            for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr  
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [39]:
optimizer=torch.optim.Adam(model.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_multinomial

In [40]:
n_batch=np.int(np.ceil(train_tokens.shape[1]/bptt))
n_batch

5667

In [41]:
model.freeze_embedding()

In [42]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(False, False)

In [43]:
learner=Learner(model,optimizer,accuracy_multinomial,device,bptt,500,0.25)

In [44]:
model.n_inp

59972

In [46]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 6.4186923856735225  0.1256440096916631
Batch:1000 5.975013018608093  0.1428456595451571
Batch:1500 5.703111585299174  0.15750889422899733
Batch:2000 5.522894755125046  0.17147609517280943
Batch:2500 5.402414136695862  0.18100878104809673
Batch:3000 5.318031203905742  0.1877163416664116
Batch:3500 5.254208151136126  0.19282243491669318
Batch:4000 5.203129969477653  0.19697120873781387
Batch:4500 5.163327643182543  0.20024992004502565
Batch:5000 5.131113591957092  0.20301871568681673
Batch:5500 5.103157820268111  0.20546087148895656
Epoch:0 Loss:5.086213410995549
Batch:500 4.631790215492249  0.24965495628118514
Epoch:0 Loss:5.086213410995549 Accuracy:0.20681632462081542 Loss:4.622790627122115 Accuracy:0.24897340615020033


In [47]:
learner.run_epochs(train_tokens,valid_tokens,2)

Batch:500 4.822180908203125  0.23166522961854935
Batch:1000 4.816756655216217  0.23130953197181225
Batch:1500 4.8144236154556275  0.2312667663494746
Batch:2000 4.811392104864121  0.231457393810153
Batch:2500 4.808410526275635  0.23159412576556207
Batch:3000 4.807046829064687  0.23170385818680128
Batch:3500 4.8042004195622035  0.231917307811124
Batch:4000 4.8003717069625855  0.23227731432765722
Batch:4500 4.798418300098843  0.23243337718645732
Batch:5000 4.797634657287598  0.23259801009893416
Batch:5500 4.795557430527427  0.23288895350423727
Epoch:0 Loss:4.793212692277114
Batch:500 4.593199374198914  0.2548681422173977
Epoch:0 Loss:4.793212692277114 Accuracy:0.23301714043248878 Loss:4.584325109752552 Accuracy:0.2541598856591722
Batch:500 4.781205368041992  0.23690770462155342
Batch:1000 4.77642805814743  0.23609374105930328
Batch:1500 4.776681967099508  0.23588988294204077
Batch:2000 4.775143810987473  0.23597905515134335
Batch:2500 4.773270655632019  0.23597246767878532
Batch:3000 4.77

In [48]:
for param_group in optimizer.param_groups:
    param_group['lr']=1e-3

In [49]:
learner.run_epochs(train_tokens,valid_tokens,4)

Batch:500 4.733539869308472  0.24251291117072105
Batch:1000 4.719546927928924  0.24291519297659397
Batch:1500 4.7128190304438276  0.2432057686348756
Batch:2000 4.707510038852692  0.24350628813356162
Batch:2500 4.70116847076416  0.24382904130220412
Batch:3000 4.697382926940918  0.24412565616269907
Batch:3500 4.693041769163949  0.2444696547644479
Batch:4000 4.687445062637329  0.24490681042149662
Batch:4500 4.683736782709757  0.24518179152078098
Batch:5000 4.681049967861176  0.24541851072609425
Batch:5500 4.6771268238587815  0.24581626212867824
Epoch:0 Loss:4.67342630883851
Batch:500 4.467592614173889  0.2683994608819485
Epoch:0 Loss:4.67342630883851 Accuracy:0.2460255659976863 Loss:4.458752728725354 Accuracy:0.26772132413714317
Batch:500 4.661553425788879  0.24950868251919747
Batch:1000 4.657487545967102  0.24878116242587567


KeyboardInterrupt: 

In [ ]:
for param_group in optimizer.param_groups:
    param_group['lr']=5e-4

In [ ]:
learner.run_epochs(train_tokens,valid_tokens,4)

In [46]:
torch.save(model.state_dict(),f'{PATH}/inter/varybptt_model_state_dict_freeze')
torch.save(optimizer.state_dict(),f'{PATH}/inter/varybptt_learner_state_dict_freeze')

In [48]:
#model.load_state_dict(torch.load(f'{PATH}/inter/varybptt_model_state_dict'))
#optimizer.load_state_dict(torch.load(f'{PATH}/inter/varybptt_learner_state_dict'))

In [ ]:
learner.run_epochs(train_tokens,valid_tokens,5)

In [47]:
model.unfreeze_embedding()

In [49]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(True, True)

In [50]:
learner.run_epochs(train_tokens,valid_tokens,5)

Batch:500 4.806624510765076  0.24196685341000557
Batch:1000 4.757751633644104  0.24493252439796925
Batch:1500 4.726561160087585  0.24686977541446686
Batch:2000 4.701271289825439  0.24833283469825984
Batch:2500 4.67970911693573  0.24951352741122246
Batch:3000 4.6627134453455605  0.2503829180995623
Batch:3500 4.646721647671291  0.2512773364995207
Batch:4000 4.63151394546032  0.2521417151428759
Batch:4500 4.61914976978302  0.2528052290976048
Batch:5000 4.608692124557495  0.25340674981176853
Batch:5500 4.597938516790217  0.25405117465149274
Epoch:0 Loss:4.590262610365019
Batch:500 4.204100080490113  0.2859588055610657
Epoch:0 Loss:4.590262610365019 Accuracy:0.25438765124873686 Loss:4.196222264230537 Accuracy:0.2853120703256111
Batch:500 4.470796348571778  0.26236842864751814
Batch:1000 4.461713090896606  0.2621190260797739
Batch:1500 4.457737143834432  0.26209192782640456
Batch:2000 4.4526124441623685  0.26223397482186556
Batch:2500 4.447095703887939  0.26240997207760813
Batch:3000 4.44378

In [51]:
torch.save(model.state_dict(),f'{PATH}/inter/varybptt_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/varybptt_learner_state_dict')

In [52]:
learner.run_epochs(train_tokens,valid_tokens,4)

Batch:500 4.272457369804382  0.27471615970134733
Batch:1000 4.267559949874878  0.2740515992343426
Batch:1500 4.267114808082581  0.273838185886542
Batch:2000 4.266339130401612  0.27369468961656096
Batch:2500 4.264564046478272  0.2736269864797592
Batch:3000 4.264817505757014  0.27352512232462567
Batch:3500 4.263663566112518  0.27356928267223496
Batch:4000 4.2613649269938465  0.27369629619270563
Batch:4500 4.2601746790674  0.2737410924302207
Batch:5000 4.259788099336624  0.273805964243412
Batch:5500 4.258415932611985  0.27396675740317866
Epoch:0 Loss:4.256667076612103
Batch:500 3.9689039225578306  0.3013489162325859
Epoch:0 Loss:4.256667076612103 Accuracy:0.273989359056968 Loss:3.961688128004424 Accuracy:0.30059371971817866
Batch:500 4.253802610397339  0.27625672483444214
Batch:1000 4.250454862594604  0.27541374777257444
Batch:1500 4.250013441085815  0.27510657292604446
Batch:2000 4.249407567024231  0.2749712665230036
Batch:2500 4.247304359817505  0.27495967178344727
Batch:3000 4.24719579

In [53]:
torch.save(model.state_dict(),f'{PATH}/inter/varybptt_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/varybptt_learner_state_dict')

In [54]:
learner.run_epochs(train_tokens,valid_tokens,2)

Batch:500 4.215587013244629  0.2790443264245987
Batch:1000 4.212339537620545  0.2783490325063467
Batch:1500 4.21223117129008  0.2779750633140405
Batch:2000 4.212278420805931  0.2777671938613057
Batch:2500 4.2109540965080265  0.27770000673532486
Batch:3000 4.211227813800176  0.2775996452718973
Batch:3500 4.210249499525343  0.27761185379964964
Batch:4000 4.20857752674818  0.2777071903422475
Batch:4500 4.207921216752794  0.27771874876817065
Batch:5000 4.208143178653717  0.2777416648209095
Batch:5500 4.207130738258362  0.27786863873221657
Epoch:0 Loss:4.205581469679631
Batch:500 3.912629920005798  0.305733531832695
Epoch:0 Loss:4.205581469679631 Accuracy:0.2779049221745413 Loss:3.905625071822171 Accuracy:0.3050169282172475
Batch:500 4.207696530342102  0.27951427340507506
Batch:1000 4.2041611847877505  0.27889167445898055
Batch:1500 4.203369742393494  0.2785125058889389
Batch:2000 4.202742716908455  0.27842603901028634
Batch:2500 4.201797800922394  0.2782968729197979
Batch:3000 4.2017882812

In [55]:
np.exp(3.9), np.exp(3.89)

(49.40244910553017, 48.91088652373189)

In [2]:
import numpy as np

In [3]:
np.exp(4.19)

66.02279096040995

In [56]:
torch.save(model.state_dict(),f'{PATH}/inter/varybptt_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/varybptt_learner_state_dict')

In [57]:
torch.save (model,f'{PATH}/inter/varybptt_model_awd_lstm')


/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type language_model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [58]:
torch.save (optimizer,f'{PATH}/inter/varybptt_optimizer_awd_lstm')

In [59]:
torch.save (learner,f'{PATH}/inter/varybptt_learner_awd_lstm')

In [45]:
model=torch.load(f'{PATH}/inter/varybptt_model_awd_lstm')
optimizer=torch.load(f'{PATH}/inter/varybptt_optimizer_awd_lstm')
learner=torch.load(f'{PATH}/inter/varybptt_learner_awd_lstm')

In [47]:
learner.print_every

500

In [48]:
learner.print_every=5000

In [51]:
for param_group in optimizer.param_groups:
    param_group['lr']=5e-4

In [52]:
learner.run_epochs(train_tokens,valid_tokens,5)

Batch:5000 4.191555965566635  0.27895016501545905
Epoch:0 Loss:4.189279972732312
Epoch:0 Loss:4.189279972732312 Accuracy:0.279056860735352 Loss:3.888390183258665 Accuracy:0.30639143109891975
Batch:5000 4.184880942630768  0.2795264025002718
Epoch:1 Loss:4.182591495435438
Epoch:1 Loss:4.182591495435438 Accuracy:0.27964237154797217 Loss:3.8800614541797547 Accuracy:0.30709379854384794
Batch:5000 4.178718735313415  0.28009060700237753
Epoch:2 Loss:4.1765148642944965
Epoch:2 Loss:4.1765148642944965 Accuracy:0.2802164892245033 Loss:3.875843460670118 Accuracy:0.3074640429666358
Batch:5000 4.174020307207107  0.28042426459491254
Epoch:3 Loss:4.171807640145708
Epoch:3 Loss:4.171807640145708 Accuracy:0.28055680217117523 Loss:3.867707792841837 Accuracy:0.3082054058710734
Batch:5000 4.168868962335586  0.28084619493186475
Epoch:4 Loss:4.166873217077107
Epoch:4 Loss:4.166873217077107 Accuracy:0.2809790029380946 Loss:3.862522657217972 Accuracy:0.3087119171113679


In [53]:
for param_group in optimizer.param_groups:
    param_group['lr']=1e-3

In [54]:
learner.run_epochs(train_tokens,valid_tokens,3)

Batch:5000 4.163958197116852  0.2813019690066576
Epoch:0 Loss:4.161988633866987
Epoch:0 Loss:4.161988633866987 Accuracy:0.28140069496502756 Loss:3.8563619542159913 Accuracy:0.3091684789463664
Batch:5000 4.1607256004333495  0.28148320442438124
Epoch:1 Loss:4.1586531713490595
Epoch:1 Loss:4.1586531713490595 Accuracy:0.2816352815020318 Loss:3.851972371957709 Accuracy:0.3095562481804137
Batch:5000 4.157159119272232  0.2818359586745501
Epoch:2 Loss:4.15509404706474
Epoch:2 Loss:4.15509404706474 Accuracy:0.28194083723833996 Loss:3.8482098567999152 Accuracy:0.3098524429581382


In [55]:
for param_group in optimizer.param_groups:
    param_group['lr']=5e-4

In [56]:
learner.run_epochs(train_tokens,valid_tokens,3)

Batch:5000 4.1534087936401365  0.2821624159812927
Epoch:0 Loss:4.151401502357985
Epoch:0 Loss:4.151401502357985 Accuracy:0.28225070216188686 Loss:3.8438201752956216 Accuracy:0.3104299358678967
Batch:5000 4.150405143165588  0.2824517137289047
Epoch:1 Loss:4.148308266302205
Epoch:1 Loss:4.148308266302205 Accuracy:0.2825875951728968 Loss:3.840221481460133 Accuracy:0.31043212698026895
Batch:5000 4.147210362434387  0.28258621204793455
Epoch:2 Loss:4.145364629223815
Epoch:2 Loss:4.145364629223815 Accuracy:0.28270174358899774 Loss:3.8401112681940983 Accuracy:0.3103839294476943


In [58]:
np.exp(3.84)

46.5254744397892

In [59]:
for param_group in optimizer.param_groups:
    param_group['lr']=5e-5

In [60]:
learner.run_epochs(train_tokens,valid_tokens,5)

Batch:5000 4.143955935955048  0.2828591864913702
Epoch:0 Loss:4.141991081134684
Epoch:0 Loss:4.141991081134684 Accuracy:0.2830100399804029 Loss:3.8335480682397383 Accuracy:0.31103152706862636
Batch:5000 4.141544650888443  0.28316244994103906
Epoch:1 Loss:4.139325787286801
Epoch:1 Loss:4.139325787286801 Accuracy:0.28328755290418833 Loss:3.829367069536419 Accuracy:0.31145347419537994
Batch:5000 4.1389409245967865  0.2834029611200094
Epoch:2 Loss:4.136891617148599
Epoch:2 Loss:4.136891617148599 Accuracy:0.28350075740207037 Loss:3.8271600257076526 Accuracy:0.31175098312718635
Batch:5000 4.136695261716842  0.2836590170711279
Epoch:3 Loss:4.134605913790469
Epoch:3 Loss:4.134605913790469 Accuracy:0.2837789879018062 Loss:3.823387879123718 Accuracy:0.31225924695317636
Batch:5000 4.134734677839279  0.2838487447053194
Epoch:4 Loss:4.13263868183117
Epoch:4 Loss:4.13263868183117 Accuracy:0.2839147918542621 Loss:3.8216802617579555 Accuracy:0.31237711128815887


In [61]:
pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights_varybptt','wb'))

In [62]:
learner.run_epochs(train_tokens,valid_tokens,25)

Batch:5000 4.132158522224426  0.28400751931071283
Epoch:0 Loss:4.130148391361934
Epoch:0 Loss:4.130148391361934 Accuracy:0.2841352554543546 Loss:3.8187610376774788 Accuracy:0.31244896963072355
Batch:5000 4.130559538078308  0.28417330280840397
Epoch:1 Loss:4.128598285223609
Epoch:1 Loss:4.128598285223609 Accuracy:0.2842837788839621 Loss:3.816732867102494 Accuracy:0.31269477657153844
Batch:5000 4.128701234436035  0.28426859176754954
Epoch:2 Loss:4.1267418176231425
Epoch:2 Loss:4.1267418176231425 Accuracy:0.28439697716034645 Loss:3.813723863406995 Accuracy:0.3128998342788581
Batch:5000 4.127094842720032  0.28455928462445734
Epoch:3 Loss:4.125193606277157
Epoch:3 Loss:4.125193606277157 Accuracy:0.28466668297768705 Loss:3.8120866023372426 Accuracy:0.31293663943403266
Batch:5000 4.126036881113053  0.28458673093020914
Epoch:4 Loss:4.123962987545563
Epoch:4 Loss:4.123962987545563 Accuracy:0.28469906520134824 Loss:3.811087922806565 Accuracy:0.3132183758550854
Batch:5000 4.124003568029404  0.284

In [63]:
pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights_varybptt_final','wb'))

In [64]:
torch.save(model.state_dict(),f'{PATH}/inter/varybptt_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/varybptt_learner_state_dict')
torch.save (model,f'{PATH}/inter/varybptt_model_awd_lstm')
torch.save (optimizer,f'{PATH}/inter/varybptt_optimizer_awd_lstm')
torch.save (learner,f'{PATH}/inter/varybptt_learner_awd_lstm')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type language_model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [65]:
np.exp(3.78)

43.81604173557396